In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from itertools import combinations, islice
import csv
from datetime import datetime, timedelta
import time

from itertools import islice


### EDA

In [3]:
# data source: https://www.kaggle.com/c/ee627a-2019fall/data?select=albumData2.txt
# origin of data: https://webscope.sandbox.yahoo.com/catalog.php?datatype=c&guccounter=1&guce_referrer=aHR0cHM6Ly9naXRodWIuY29tL3NhcmFueWF2c3IvTXVzaWMtUmVjb21tZW5kYXRpb25z&guce_referrer_sig=AQAAADdDVj1NcJ7l9D0AF1OwjrIchcuyq2aDD8kc4qxRk3RP-B1mQTaY0IDliV2wsC-gQw05v-d9k8v70efaNULAbemXR_upER5MDVS8mcDsU_DQJZmtcUF8Sdh7A1holj3I-8UJVcKbI65keJp44o46CL8aGp2kLYhRCUYeTXkwxv9N


data_list = glob.glob('ee627a-2019fall\*')
data_list

[]

- trainItem2.txt - the training set 
- testItem2.txt - the test set 
- sample_ submission.csv - a sample submission file in the correct format 
- trackData2.txt -- Track information formatted as: <'TrackId'>|<'AlbumId'>|<'ArtistId'>|<'Optional GenreId_1'>|...|<'Optional GenreId_k'> 
- albumData2.txt -- Album information formatted as: <'AlbumId'>|<'ArtistId'>|<'Optional GenreId_1'>|...|<'Optional GenreId_k'> 
- artistData2.txt -- Artist listing formatted as: <'ArtistId'>
- genreData2.txt -- Genre listing formatted as: <'GenreId'>



## Process for surprise

In [2]:
# process tracks data file
rows = []
for row in open('ee627a-2019fall/trainItem2.txt'):
    if '|' in row:
        cur_user = row.strip('\n').split('|')[0] # pull user ID. don't need song count
        continue # skip to the user's ratings
    row = row.strip('\n').split('\t')
    row_dict = {'user': cur_user, 
                'track': row[0], 
                'rating': int(row[1])
               }
    rows.append(row_dict)

df_train = pd.DataFrame(rows)

In [3]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate
svd = SVD()

reader = Reader(rating_scale=(0, 100))
data = Dataset.load_from_df(df_train, reader)
trainset = data.build_full_trainset()

In [14]:
df_train.info()

AttributeError: 'NoneType' object has no attribute 'info'

## SVD

In [9]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    63.0237 63.0137 62.9738 62.9827 63.0187 63.0025 0.0203  
MAE (testset)     50.2529 50.2442 50.1980 50.2079 50.2439 50.2294 0.0221  
Fit time          470.88  431.81  423.76  422.57  423.45  434.49  18.49   
Test time         29.59   25.72   28.56   28.69   26.96   27.90   1.38    


{'test_rmse': array([63.02372132, 63.01368387, 62.97381311, 62.98268164, 63.0187177 ]),
 'test_mae': array([50.25291096, 50.24423725, 50.19802517, 50.20785983, 50.2439458 ]),
 'fit_time': (470.87875986099243,
  431.8085570335388,
  423.76098585128784,
  422.57399702072144,
  423.4520380496979),
 'test_time': (29.590038537979126,
  25.724996089935303,
  28.555996894836426,
  28.688954830169678,
  26.964996576309204)}

In [5]:
%time svd.fit(trainset)


Wall time: 8min 7s


In [15]:
trainset

In [20]:
df_train

,user,track,rating
0,199808,248969,90
1,199808,2663,90
2,199808,28341,90
3,199808,42563,90
4,199808,59092,90
...,...,...,...
12403570,249011,270557,90
12403571,249011,273574,90
12403572,249011,286938,90
12403573,249011,287681,80


#### make predictions

In [25]:
svd.predict(199810, int(208019))

Prediction(uid=199810, iid=208019, r_ui=None, est=49.77060420080501, details={'was_impossible': False})

In [21]:
predictions = []
with open('ee627a-2019fall/testItem2.txt') as f:
    while True:
        next_n_lines = list(islice(f, 7))
        if not next_n_lines:
            break
        ratings = [] # reset for new user's list of songs
        cur_set = [x.strip('\n').split('|') for x in next_n_lines]
        test_user = int(cur_set[0][0]) # pull user ID. don't need song count 
        for track in cur_set[1:]:
            print(track, test_user)
            rating = svd.predict(test_user, int(track[0])).est
            rating_tup = (int(track[0]), rating)
            ratings.append(rating_tup)
        
        ratings.sort(key=lambda x:x[1]) # sort 6 songs by rating
        for i, j in enumerate(ratings):
            cur_dict = {}
            cur_dict['TrackID'] = f'{test_user}_{j[0]}'
            cur_dict['rating'] = j[1]
            if i < 3:
                cur_dict['Predictor'] = 0 
            else:
                cur_dict['Predictor'] = 1 
            predictions.append(cur_dict)
        

['208019'] 199810
['74139'] 199810
['9903'] 199810
['242681'] 199810
['18515'] 199810
['105760'] 199810
['276940'] 199812
['142408'] 199812
['130023'] 199812
['29189'] 199812
['223706'] 199812
['211361'] 199812
['188441'] 199813
['20968'] 199813
['21571'] 199813
['79640'] 199813
['184173'] 199813
['111874'] 199813
['122375'] 199814
['189043'] 199814
['122429'] 199814
['52519'] 199814
['232332'] 199814
['262193'] 199814
['64345'] 199815
['118841'] 199815
['275682'] 199815
['30062'] 199815
['258473'] 199815
['129866'] 199815
['274758'] 199816
['102153'] 199816
['183464'] 199816
['23616'] 199816
['81699'] 199816
['46627'] 199816
['83722'] 199817
['200166'] 199817
['148603'] 199817
['88745'] 199817
['144775'] 199817
['222966'] 199817
['141875'] 199818
['290905'] 199818
['59789'] 199818
['283698'] 199818
['106788'] 199818
['235999'] 199818
['202308'] 199819
['36793'] 199819
['247549'] 199819
['214280'] 199819
['123424'] 199819
['193003'] 199819
['235240'] 199820
['292341'] 199820
['126082']

['45786'] 200398
['175194'] 200398
['227719'] 200398
['207699'] 200398
['263168'] 200400
['64675'] 200400
['99158'] 200400
['133094'] 200400
['281571'] 200400
['231680'] 200400
['240498'] 200401
['91555'] 200401
['220726'] 200401
['236433'] 200401
['51468'] 200401
['244960'] 200401
['135699'] 200406
['182693'] 200406
['91014'] 200406
['191480'] 200406
['135962'] 200406
['36732'] 200406
['147604'] 200407
['175560'] 200407
['103861'] 200407
['200166'] 200407
['214172'] 200407
['7740'] 200407
['292176'] 200413
['133594'] 200413
['291958'] 200413
['59772'] 200413
['94007'] 200413
['139505'] 200413
['187953'] 200414
['210791'] 200414
['267796'] 200414
['10679'] 200414
['262192'] 200414
['284820'] 200414
['252040'] 200416
['266225'] 200416
['109787'] 200416
['77677'] 200416
['149254'] 200416
['32664'] 200416
['124070'] 200419
['128565'] 200419
['115358'] 200419
['143033'] 200419
['34290'] 200419
['101875'] 200419
['92711'] 200420
['141772'] 200420
['151596'] 200420
['685'] 200420
['291238'] 

['115550'] 200955
['22924'] 200955
['3916'] 200955
['175812'] 200955
['137107'] 200955
['91280'] 200956
['61941'] 200956
['264826'] 200956
['137479'] 200956
['83900'] 200956
['142675'] 200956
['56106'] 200958
['137468'] 200958
['193801'] 200958
['209487'] 200958
['12415'] 200958
['46756'] 200958
['56678'] 200959
['176586'] 200959
['33066'] 200959
['165296'] 200959
['166917'] 200959
['94498'] 200959
['263043'] 200961
['36652'] 200961
['53593'] 200961
['139556'] 200961
['111988'] 200961
['32231'] 200961
['145494'] 200962
['78910'] 200962
['263043'] 200962
['192439'] 200962
['4993'] 200962
['179615'] 200962
['191598'] 200970
['289153'] 200970
['132591'] 200970
['112264'] 200970
['178277'] 200970
['266176'] 200970
['143401'] 200976
['27864'] 200976
['27672'] 200976
['158665'] 200976
['166193'] 200976
['169505'] 200976
['129775'] 200977
['197995'] 200977
['33860'] 200977
['253231'] 200977
['137882'] 200977
['37960'] 200977
['161986'] 200979
['203180'] 200979
['4906'] 200979
['16498'] 200979

['143734'] 201564
['6848'] 201564
['252592'] 201564
['40057'] 201564
['158757'] 201564
['71480'] 201566
['50411'] 201566
['169706'] 201566
['237679'] 201566
['58072'] 201566
['231521'] 201566
['271794'] 201567
['15951'] 201567
['81709'] 201567
['97522'] 201567
['141077'] 201567
['96378'] 201567
['241059'] 201568
['118195'] 201568
['221040'] 201568
['228676'] 201568
['74098'] 201568
['4220'] 201568
['216738'] 201570
['26132'] 201570
['158554'] 201570
['221711'] 201570
['70378'] 201570
['290794'] 201570
['112760'] 201572
['22156'] 201572
['6116'] 201572
['94893'] 201572
['184702'] 201572
['134431'] 201572
['127986'] 201575
['214172'] 201575
['249257'] 201575
['210733'] 201575
['175981'] 201575
['175211'] 201575
['52572'] 201581
['201981'] 201581
['211592'] 201581
['210990'] 201581
['131564'] 201581
['294289'] 201581
['229340'] 201582
['196733'] 201582
['235676'] 201582
['67687'] 201582
['43077'] 201582
['289345'] 201582
['175306'] 201583
['134816'] 201583
['285882'] 201583
['156151'] 201

['141376'] 202278
['64749'] 202278
['130074'] 202279
['286433'] 202279
['57701'] 202279
['257185'] 202279
['26867'] 202279
['224016'] 202279
['134527'] 202282
['257663'] 202282
['111748'] 202282
['124486'] 202282
['62493'] 202282
['240094'] 202282
['125702'] 202284
['193382'] 202284
['293408'] 202284
['53925'] 202284
['136610'] 202284
['237564'] 202284
['257990'] 202285
['225414'] 202285
['51665'] 202285
['228745'] 202285
['228535'] 202285
['53446'] 202285
['177062'] 202289
['145338'] 202289
['10335'] 202289
['12197'] 202289
['179788'] 202289
['219713'] 202289
['186500'] 202292
['175714'] 202292
['100384'] 202292
['95704'] 202292
['166592'] 202292
['224108'] 202292
['55417'] 202295
['153923'] 202295
['101201'] 202295
['126713'] 202295
['116957'] 202295
['239582'] 202295
['51675'] 202296
['157403'] 202296
['194883'] 202296
['78132'] 202296
['65601'] 202296
['162662'] 202296
['107168'] 202298
['292863'] 202298
['226836'] 202298
['49153'] 202298
['1754'] 202298
['30845'] 202298
['118191']

...

['71041'] 246058
['146808'] 246064
['234644'] 246064
['209370'] 246064
['17892'] 246064
['186375'] 246064
['223414'] 246064
['261846'] 246074
['140334'] 246074
['72839'] 246074
['114673'] 246074
['234345'] 246074
['265804'] 246074
['150618'] 246076
['211241'] 246076
['101294'] 246076
['568'] 246076
['192100'] 246076
['137071'] 246076
['91395'] 246079
['53592'] 246079
['72192'] 246079
['209370'] 246079
['129274'] 246079
['152465'] 246079
['248692'] 246089
['186679'] 246089
['146430'] 246089
['30970'] 246089
['118999'] 246089
['187322'] 246089
['153923'] 246091
['5844'] 246091
['243020'] 246091
['206126'] 246091
['49793'] 246091
['192136'] 246091
['40373'] 246096
['94459'] 246096
['156375'] 246096
['197607'] 246096
['165413'] 246096
['148406'] 246096
['98871'] 246100
['239701'] 246100
['281016'] 246100
['94829'] 246100
['135622'] 246100
['115126'] 246100
['273400'] 246103
['72013'] 246103
['253912'] 246103
['79453'] 246103
['265067'] 246103
['92035'] 246103
['35604'] 246105
['200186'] 24

['243813'] 246698
['38202'] 246698
['241460'] 246698
['24407'] 246698
['234206'] 246698
['285549'] 246699
['139002'] 246699
['165644'] 246699
['188123'] 246699
['252285'] 246699
['180461'] 246699
['215106'] 246704
['91782'] 246704
['287949'] 246704
['73744'] 246704
['214317'] 246704
['264511'] 246704
['190622'] 246705
['6747'] 246705
['140802'] 246705
['176821'] 246705
['261013'] 246705
['226946'] 246705
['289304'] 246708
['175481'] 246708
['224544'] 246708
['184895'] 246708
['140843'] 246708
['286770'] 246708
['209793'] 246711
['32226'] 246711
['152626'] 246711
['211123'] 246711
['59890'] 246711
['73557'] 246711
['153972'] 246712
['8244'] 246712
['245114'] 246712
['156112'] 246712
['98103'] 246712
['176676'] 246712
['210982'] 246714
['244792'] 246714
['110099'] 246714
['153923'] 246714
['273211'] 246714
['86212'] 246714
['57118'] 246717
['146263'] 246717
['22166'] 246717
['125254'] 246717
['88999'] 246717
['88589'] 246717
['199553'] 246722
['277360'] 246722
['175457'] 246722
['136504'

['219934'] 247397
['93084'] 247397
['117833'] 247397
['66859'] 247397
['225414'] 247397
['205398'] 247397
['82581'] 247398
['20036'] 247398
['131155'] 247398
['85180'] 247398
['9464'] 247398
['118861'] 247398
['130023'] 247400
['230842'] 247400
['212592'] 247400
['182009'] 247400
['234441'] 247400
['235014'] 247400
['31170'] 247402
['204632'] 247402
['110205'] 247402
['72387'] 247402
['220242'] 247402
['75564'] 247402
['138741'] 247404
['131863'] 247404
['99448'] 247404
['269530'] 247404
['271466'] 247404
['295936'] 247404
['167977'] 247411
['187117'] 247411
['123997'] 247411
['143428'] 247411
['171534'] 247411
['159976'] 247411
['166025'] 247412
['169305'] 247412
['144357'] 247412
['45127'] 247412
['210791'] 247412
['214994'] 247412
['9838'] 247413
['253519'] 247413
['61826'] 247413
['140019'] 247413
['259248'] 247413
['249919'] 247413
['46482'] 247417
['242047'] 247417
['210744'] 247417
['8313'] 247417
['182786'] 247417
['160555'] 247417
['73588'] 247425
['134958'] 247425
['118076'] 

['193856'] 248128
['65935'] 248129
['60913'] 248129
['171373'] 248129
['119280'] 248129
['28053'] 248129
['138384'] 248129
['36181'] 248130
['190420'] 248130
['197914'] 248130
['116613'] 248130
['215158'] 248130
['272845'] 248130
['173538'] 248131
['67602'] 248131
['27852'] 248131
['294134'] 248131
['220488'] 248131
['14542'] 248131
['75813'] 248133
['153923'] 248133
['6064'] 248133
['291976'] 248133
['45685'] 248133
['202160'] 248133
['64045'] 248134
['46661'] 248134
['227199'] 248134
['247711'] 248134
['21548'] 248134
['233572'] 248134
['183841'] 248135
['179603'] 248135
['69175'] 248135
['21365'] 248135
['218730'] 248135
['128323'] 248135
['138168'] 248136
['108977'] 248136
['282608'] 248136
['189941'] 248136
['65621'] 248136
['264787'] 248136
['277773'] 248137
['232559'] 248137
['85777'] 248137
['231072'] 248137
['160321'] 248137
['169932'] 248137
['152518'] 248138
['140128'] 248138
['41843'] 248138
['69821'] 248138
['23288'] 248138
['226836'] 248138
['53085'] 248140
['222878'] 248

['175670'] 248766
['51133'] 248769
['48000'] 248769
['105075'] 248769
['288091'] 248769
['64665'] 248769
['256840'] 248769
['83614'] 248770
['152491'] 248770
['14136'] 248770
['35580'] 248770
['87328'] 248770
['146446'] 248770
['272638'] 248773
['246130'] 248773
['259064'] 248773
['85262'] 248773
['239591'] 248773
['123328'] 248773
['101360'] 248775
['239100'] 248775
['21529'] 248775
['136523'] 248775
['179535'] 248775
['269998'] 248775
['141660'] 248776
['107737'] 248776
['155798'] 248776
['232769'] 248776
['101871'] 248776
['151143'] 248776
['275379'] 248780
['88852'] 248780
['125019'] 248780
['201393'] 248780
['121733'] 248780
['193878'] 248780
['162199'] 248782
['29189'] 248782
['76222'] 248782
['214776'] 248782
['58082'] 248782
['199558'] 248782
['253265'] 248785
['257270'] 248785
['221998'] 248785
['183971'] 248785
['133769'] 248785
['241491'] 248785
['204355'] 248787
['176515'] 248787
['62157'] 248787
['164212'] 248787
['77781'] 248787
['122375'] 248787
['121293'] 248788
['14146

In [19]:
preditions_df = pd.DataFrame(predictions)
preditions_df

,TrackID,rating,Predictor
0,199810_208019,49.770604,0
1,199810_74139,49.770604,0
2,199810_9903,49.770604,0
3,199810_242681,49.770604,1
4,199810_18515,49.770604,1
...,...,...,...
119995,249010_72192,49.770604,0
119996,249010_86104,49.770604,0
119997,249010_186634,49.770604,1
119998,249010_293818,49.770604,1


## KNN

In [28]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)



Computing the cosine similarity matrix...


MemoryError: Unable to allocate 652. GiB for an array with shape (295799, 295799) and data type float64

In [12]:
from surprise import KNNWithMeans

# To use item-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)



Computing the cosine similarity matrix...


MemoryError: Unable to allocate 18.0 GiB for an array with shape (49204, 49204) and data type float64

In [ ]:
prediction = algo.predict('E', 2) # E=user, 2=movie

# Graveyard

## Process for networkx

In [4]:
df_tracks = pd.DataFrame()

# process tracks data file
for row in open('ee627a-2019fall/trackData2.txt'):
    row = row.strip('\n').split('|')
    G.add_node(row[0], attr = {"node_type": "track"}) # add track node
    G.add_edge(row[0], row[1], weight=100) # connect track to album
    G.add_node(row[1], attr = {"node_type": "album"}) # add album node
    G.add_node(row[2], attr = {"node_type": "artist"}) # add artist node
    if len(row) > 3:
        for genre in row[3:]:
            G.add_node(genre, attr = {"node_type": "genre"}) # add genre node
            G.add_edge(row[0], genre, weight=100) # connect each genre to the track

In [5]:
# process albums data file
for row in open('ee627a-2019fall/albumData2.txt'):
    row = row.strip('\n').split('|')
    G.add_edge(row[0], row[1], weight=100) # connect album to artist
    if len(row) > 2:
        for genre in row[2:]:
            G.add_node(genre, attr = {"node_type": "genre"}) # add genre node
            G.add_edge(row[0], genre, weight=100) # connect each genre to the album

In [6]:
# process training user data file
for row in open('ee627a-2019fall/trainItem2.txt'):
    if '|' in row:
        cur_user = row.strip('\n').split('|')[0] # pull user ID. don't need song count
        G.add_node(cur_user, attr = {"node_type": "user"}) # add user node
        continue # skip to the user's ratings
    row = row.strip('\n').split('\t')
    G.add_edge(cur_user, row[0], weight=int(row[1])) # connect user to song with rating as edge weight

In [7]:
print(len(G.nodes))
print(len(G.edges)) # 956820
print(len([1 for cc in nx.connected_components(G)]))


296101
13342506
5


### G2 selects the largest subgraph. appears to just drop 4 disconnected nodes
not currently needed because switched to astar, which is ok with weakly connected graphs  
don't want to process if not necessary because the 2nd graph takes about 1.5GB of memory

### Processing attempt 1. estimated time to completion of 20 days

### Processing attempt 2. parallelized - ETC 6 days

In [17]:
df_1strun = pd.read_csv('recommendations-Copy1.csv')
df_1strun['TrackID'] = df_1strun['TrackID'].str.split("_", n = 1, expand = True)

completed_users = df_1strun.TrackID.unique()

In [18]:
def write_user(package, outfile='./recommendations.csv'):  
    with open(outfile, 'a',encoding='utf-8-sig', newline='\n') as g:
        keys = package[0].keys() # get keys off first rating
        dict_writer = csv.DictWriter(g, keys)
        dict_writer.writerows(package)


def process_user(i, infile='ee627a-2019fall/testItem2.txt'):
    with open(infile, "r") as f:
        lines_gen = islice(f, i*7, (i+1)*7) # get line index for userID & their last target song
        cur_set = [x.strip('\n').split('|') for x in lines_gen]
    test_user = cur_set[0][0] # pull user ID. don't need song count  199810_208019
    if test_user in completed_users:
        return None
    distances = []
    for song in cur_set[1:]:
        dist=nx.astar_path_length(G, str(test_user), \
                                     str(song[0]), \
                                     weight='weight') # distance from user to target song
        distances.append((f'{test_user}_{song[0]}', dist))
    distances.sort(key=lambda x:x[1])
    predictions = []
    for i, j in enumerate(distances):
        cur_dict = {}
        cur_dict['TrackID'] = j[0]
        if i < 3:
            cur_dict['Predictor'] = 1 # sorted ascending, smaller distance means closer & should be recommended
        else:
            cur_dict['Predictor'] = 0
        predictions.append(cur_dict)

    return predictions

class Worker(threading.Thread):

    def __init__(self, q, i, *args, **kwargs):
        self.q = q
        self.i = i
        super().__init__(*args, **kwargs)

            
    def run(self):
        while True:
            try:
                j = self.q.get(timeout=3)  # 3s timeout
            except queue.Empty:
                return
            

            predictions_processed = process_user(j)
            if predictions_processed: # none will be returned if user already completed so we can skip
                write_user(predictions_processed)

            self.q.task_done()


def mt_user_predictions(infile, outfile):
    print('start time: {}'.format(datetime.now().strftime("%Y-%m-%d-%H.%M.%S")))
    start_time = time.time()

    # n_threads= int(args[0])
    n_threads = 30
    ################### add desired output columns
    with open(outfile ,'w') as oufl, open(infile, 'r', encoding='utf-8') as infl:
        oufl.write('TrackID,Predictor')
        row_count = sum(1 for row in infl)
    print('total rows:', row_count) 
    
    q = queue.Queue()
    
    n_users = int((row_count)/7)
    with open(infile) as f:
        for i in range(n_users):
            q.put_nowait(i)
    print(f'[{n_users}] users in target file. passing indices to user multithreading')
    for _ in range(n_threads):
        Worker(q, _).start()
        time.sleep(1)
    q.join()
    
    print('finished. end time: {}'.format(datetime.now().strftime("%Y-%m-%d-%H.%M.%S")))
    print('completed in {}'.format(timedelta(seconds=int(time.time() - start_time))))

    ### multithreading doesn't help. this is PCU heavy. dumby....
# mt_user_predictions(infile='ee627a-2019fall/testItem2.txt', outfile='./recommendations.csv')            

##### switch to multiprocessing

In [19]:
infile='ee627a-2019fall/testItem2.txt'
outfile='./recommendations.csv'

################### add desired output columns
with open(outfile ,'w') as oufl, open(infile, 'r', encoding='utf-8') as infl:
    oufl.write('TrackID,Predictor\n')
    row_count = sum(1 for row in infl)
print('total rows:', row_count) 
n_users = int((row_count)/7)
tasks = list(range(n_users))

pbar = tqdm.tqdm(total=len(tasks))

def run_mp(j):            
    predictions_processed = process_user(j)
    write_user(predictions_processed)
    pbar.update(1)
 
print('start time: {}'.format(datetime.now().strftime("%Y-%m-%d-%H.%M.%S")))
start_time = time.time()


num_workers = mp.cpu_count()
print('num workers avail:', num_workers)

pool = mp.Pool()
pool.imap_unordered(run_mp, tasks)



  0%|          | 0/20000 [00:00<?, ?it/s]

total rows: 140000
start time: 2022-03-08-17.17.53
num workers avail: 16


  0%|          | 41/20000 [4:32:23<2584:08:17, 466.10s/it]

In [ ]:
""" try 1  
pool = mp.Pool(num_workers)
for _ in tqdm.tqdm(pool.map(func=run_mp, iterable=tasks, chunksize=20), total=len(tasks)):
    pass

result = pool.map(func=run_mp, iterable=tasks, chunksize=2)
"""

""" try 2
# pool = mp.Pool()
# pbar = tqdm.tqdm(total=len(tasks))

# pool.imap_unordered(run_mp, tasks)
"""

"""try 3
num_workers = mp.cpu_count()
print('num workers avail:', num_workers)
pbar = tqdm.tqdm(total=len(tasks))

with mp.Pool(num_workers) as p:
    for i, r in enumerate(p.imap_unordered(run_mp, tasks)):

        pbar.update(i)

pbar.close()
"""
#### at 12:27, 14hr eta means done before 3pm
#### at 10:31, 1532125it [10:08:20, 146.29it/s], 10,500 lines written after 10 hrs. ~1000/hr = 140hrs

### Docker setup
https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html#docker

start image:
sudo docker run --rm --gpus all nvidia/cuda:11.0-base nvidia-smi
alternate
sudo nvidia-docker run -it -v ~/gits/nw-msds458/final:/wkg nvidia/cuda:11-base nvidia-smi

------
sudo docker run \
    --rm \
    -it \
    --gpus all \
    -v ~/gits/nw-msds458/final:/wkg \
    -e EXTRA_APT_PACKAGES="vim nano" \
    -e EXTRA_CONDA_PACKAGES="jq" \
    -e EXTRA_PIP_PACKAGES="networkx" \
    -p 5555:8888 \
    -p 8787:8787 \
    -p 8786:8786 \
    rapidsai/rapidsai:22.02-cuda11.0-runtime-ubuntu18.04-py3.8


In [ ]:
#####################################################################################################

## Graveyard  
other distance measure attemps. even slower  
also post processing that isn't necessary if i write to file

In [6]:
listy = [('asdf', 3), ('sdfg', 2), ('dfgh', 80), ('fghj', 1), ('ghjk', 34), ('hjkl', 23)]
listy.sort(key=lambda x:x[1])
print(listy)
predicitons = []
for i, j in enumerate(listy):
    cur_dict = {}
#     cur_dict['TrackID'] = f'{test_user}_{song}'
    cur_dict['TrackID'] = f'{j[0]}'
    if i < 3:
        cur_dict['Predictor'] = 1 # sorted ascending, smaller distance means closer & should be recommended
    else:
        cur_dict['Predictor'] = 0
    predicitons.append(cur_dict)
print(predicitons)

[('fghj', 1), ('sdfg', 2), ('asdf', 3), ('hjkl', 23), ('ghjk', 34), ('dfgh', 80)]
[{'TrackID': 'fghj', 'Predictor': 0}, {'TrackID': 'sdfg', 'Predictor': 0}, {'TrackID': 'asdf', 'Predictor': 0}, {'TrackID': 'hjkl', 'Predictor': 1}, {'TrackID': 'ghjk', 'Predictor': 1}, {'TrackID': 'dfgh', 'Predictor': 1}]


In [9]:
# # connect all unconnected components with 0 weight so graph is strongly connected
# combs = list(combinations(list(G.nodes), 2))
# for comb in combs:
#     if not G.has_edge(comb[0], comb[1]):
#         G.add_edge(comb[0], comb[1], weight=0)

### memory error

In [10]:
# too big
# subax1 = plt.subplot(121)
# nx.draw(G, with_labels=True)

In [11]:
%time nx.shortest_path(G2, '1', '214765')

CPU times: user 28 µs, sys: 1 µs, total: 29 µs
Wall time: 31 µs


['1', '214765']

In [12]:
try:
    %time length=nx.astar_path_length(G, '1', '214765')
except:
    print('switching to subgraph')
    %time length=nx.astar_path_length(G2, '1', '214765')

# print(path['1']['214765'])
print(length)


CPU times: user 48 µs, sys: 1 µs, total: 49 µs
Wall time: 51 µs
100


In [ ]:
%time nx.resistance_distance(G2, '1', '214765')

In [ ]:
try:
    %time length2=nx.dijkstra_path_length(G, '1', '214765')
except:
    print('switching to subgraph')
    %time length2=nx.dijkstra_path_length(G2, '1', '214765')

# print(path['1']['214765'])
print(length2)


In [ ]:
# %time nx.dijkstra_path(G2, '1', '214765', weight='weight')

In [ ]:
# try:
#     %time path=nx.all_pairs_dijkstra_path(G)
# except:
#     print('switching to subgraph')
#     %time path=nx.all_pairs_dijkstra_path(G2)

# print(path['1']['214765'])


In [ ]:
# # is this cell necessary? process df in next for predicting 1/0
# with f as open('ee627a-2019fall\\testItem2.txt'):



In [ ]:
# submit to: https://www.kaggle.com/c/ee627a-2019fall/submit
df_submission = pd.read_csv('ee627a-2019fall\\sample_submission.csv')
df_submission[['user', 'track']] = df_submission['TrackID'].str.split('_', 1, expand=True)

df_submission